# Executive Summary

The purpose of this report is to calculate and analyze the loyalty points of players on ABC's online gaming platform for the month of October 2022. This report covers the detailed calculation of loyalty points for specific slots, overall ranking of players, bonus allocation for top players, and an evaluation of the current loyalty point formula with suggestions for improvement.

# Introduction
ABC is a real-money online gaming company providing multiplayer games such as Ludo. Players can register, deposit money, play games, and withdraw their winnings from the platform. The company incentivizes player activity through a loyalty points system based on deposits, withdrawals, and gameplay. This report aims to calculate loyalty points for players, rank them based on their points, allocate bonuses to top players, and evaluate the fairness of the loyalty point formula.

# Data Description

The data for this analysis is extracted from three tables: User_Gameplay, Deposit, and Withdrawal.

##### 1. User_Gameplay Table:

User_ID (Varchar(50)): Unique identifier for the player

Games_Played (Varchar(50)): Number of games played

Datetime (Datetime): Timestamp of the gameplay activity

##### 2. Deposit Table:

User_Id (Varchar(50)): Unique identifier for the player

Datetime (Datetime): Timestamp of the deposit activity

Amount (Varchar(50)): Amount deposited

##### 3. Withdrawal Table:

User_Id (Varchar(50)): Unique identifier for the player

Datetime (Datetime): Timestamp of the withdrawal activity

Amount (Varchar(50)): Amount withdrawn

# Methodology
The loyalty points are calculated based on the following formula:

##### Loyalty Point = (0.01 * deposit) + (0.005 * Withdrawal amount) + (0.001 * (maximum of (#deposit - #withdrawal) or 0)) + (0.2 * Number of games played)


#### The methodology involves:

- Loading the data from the three tables
- Calculating loyalty points for specified slots

- Calculating overall loyalty points for the month of October

- Ranking players based on their loyalty points

- Allocating bonuses to the top 50 players

- Evaluating the current loyalty point formula


# Part A: Calculating Loyalty Points

#### 1. Player-wise Loyalty Points for all the Slots

In [ ]:
!pip install pandas sqlalchemy pyodbc

In [63]:
import pandas as pd
from sqlalchemy import create_engine

In [68]:
engine = create_engine('mssql+pyodbc://@DESKTOP-V2B514I/Gaming_data?driver=ODBC+Driver+17+for+SQL+Server')

In [69]:
print(engine)

Engine(mssql+pyodbc://@DESKTOP-V2B514I/Gaming_data?driver=ODBC+Driver+17+for+SQL+Server)


In [72]:
loyalty_points = """
WITH Slots AS (
    SELECT '2022-10-02 00:00:00' AS StartDateTime, '2022-10-02 12:00:00' AS EndDateTime, 'Slot1_02Oct' AS SlotLabel UNION ALL
    SELECT '2022-10-16 12:00:00', '2022-10-17 00:00:00', 'Slot2_16Oct' UNION ALL
    SELECT '2022-10-18 00:00:00', '2022-10-18 12:00:00', 'Slot1_18Oct' UNION ALL
    SELECT '2022-10-26 12:00:00', '2022-10-27 00:00:00', 'Slot2_26Oct'
),
Deposits AS (
    SELECT s.SlotLabel, d.User_ID, SUM(CAST(d.Amount AS DECIMAL(10, 2))) AS Total_Deposit, COUNT(*) AS Num_Deposits
    FROM Deposit d
    JOIN Slots s ON d.Datetime >= s.StartDateTime AND d.Datetime < s.EndDateTime
    GROUP BY s.SlotLabel, d.User_ID
),
Withdrawals AS (
    SELECT s.SlotLabel, w.User_ID, SUM(CAST(w.Amount AS DECIMAL(10, 2))) AS Total_Withdrawal, COUNT(*) AS Num_Withdrawals
    FROM Withdrawal w
    JOIN Slots s ON w.Datetime >= s.StartDateTime AND w.Datetime < s.EndDateTime
    GROUP BY s.SlotLabel, w.User_ID
),
GamesPlayed AS (
    SELECT s.SlotLabel, g.User_ID, SUM(CAST(g.Games_Played AS INT)) AS Total_Games_Played
    FROM User_Gameplay g
    JOIN Slots s ON g.Datetime >= s.StartDateTime AND g.Datetime < s.EndDateTime
    GROUP BY s.SlotLabel, g.User_ID
)
SELECT
    s.SlotLabel,
    COALESCE(d.User_ID, w.User_ID, g.User_ID) AS User_ID,
    COALESCE(Total_Deposit, 0) AS Total_Deposit,
    COALESCE(Total_Withdrawal, 0) AS Total_Withdrawal,
    COALESCE(Total_Games_Played, 0) AS Total_Games_Played,
    (0.01 * COALESCE(Total_Deposit, 0)) + 
    (0.005 * COALESCE(Total_Withdrawal, 0)) + 
    (0.001 * GREATEST(COALESCE(d.Num_Deposits, 0) - COALESCE(w.Num_Withdrawals, 0), 0)) + 
    (0.2 * COALESCE(Total_Games_Played, 0)) AS Loyalty_Points
FROM Slots s
LEFT JOIN Deposits d ON s.SlotLabel = d.SlotLabel
LEFT JOIN Withdrawals w ON s.SlotLabel = w.SlotLabel AND d.User_ID = w.User_ID
LEFT JOIN GamesPlayed g ON s.SlotLabel = g.SlotLabel AND COALESCE(d.User_ID, w.User_ID) = g.User_ID
ORDER BY s.SlotLabel, User_ID;
"""

df = pd.read_sql(loyalty_points, engine)
df.head(10)


,SlotLabel,User_ID,Total_Deposit,Total_Withdrawal,Total_Games_Played,Loyalty_Points
0,Slot1_02Oct,10,2000.0,0.0,13,22.601
1,Slot1_02Oct,104,10000.0,0.0,1,100.201
2,Slot1_02Oct,106,648.0,10000.0,0,56.481
3,Slot1_02Oct,11,2000.0,0.0,15,23.001
4,Slot1_02Oct,120,2000.0,0.0,0,20.001
5,Slot1_02Oct,122,1000.0,0.0,0,10.001
6,Slot1_02Oct,125,500.0,0.0,0,5.001
7,Slot1_02Oct,131,12000.0,0.0,5,121.002
8,Slot1_02Oct,136,2000.0,0.0,2,20.401
9,Slot1_02Oct,14,100.0,0.0,0,1.001


# Player ranking based on loyalty points

In [76]:
player_ranking = """
WITH MonthlyDeposits AS (
    SELECT d.User_ID, SUM(CAST(d.Amount AS DECIMAL(10, 2))) AS Total_Deposit, COUNT(*) AS Num_Deposits
    FROM Deposit d
    WHERE CONVERT(DATE, d.Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY d.User_ID
),
MonthlyWithdrawals AS (
    SELECT w.User_ID, SUM(CAST(w.Amount AS DECIMAL(10, 2))) AS Total_Withdrawal, COUNT(*) AS Num_Withdrawals
    FROM Withdrawal w
    WHERE CONVERT(DATE, w.Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY w.User_ID
),
MonthlyGamesPlayed AS (
    SELECT g.User_ID, SUM(CAST(g.Games_Played AS INT)) AS Total_Games_Played
    FROM User_Gameplay g
    WHERE CONVERT(DATE, g.Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY g.User_ID
),
TotalLoyaltyPoints AS (
    SELECT
        COALESCE(d.User_ID, w.User_ID, g.User_ID) AS User_ID,
        COALESCE(Total_Deposit, 0) AS Total_Deposit,
        COALESCE(Total_Withdrawal, 0) AS Total_Withdrawal,
        COALESCE(Total_Games_Played, 0) AS Total_Games_Played,
        (0.01 * COALESCE(Total_Deposit, 0)) + 
        (0.005 * COALESCE(Total_Withdrawal, 0)) + 
        (0.001 * GREATEST(COALESCE(d.Num_Deposits, 0) - COALESCE(w.Num_Withdrawals, 0), 0)) + 
        (0.2 * COALESCE(Total_Games_Played, 0)) AS Total_Loyalty_Points
    FROM MonthlyDeposits d
    LEFT JOIN MonthlyWithdrawals w ON d.User_ID = w.User_ID
    LEFT JOIN MonthlyGamesPlayed g ON COALESCE(d.User_ID, w.User_ID) = g.User_ID
)
SELECT
    User_ID,
    Total_Deposit,
    Total_Withdrawal,
    Total_Games_Played,
    Total_Loyalty_Points,
    RANK() OVER (ORDER BY Total_Loyalty_Points DESC, Total_Games_Played DESC) AS Rank
FROM TotalLoyaltyPoints;
"""
df = pd.read_sql(player_ranking, engine)
df.head(10)

,User_ID,Total_Deposit,Total_Withdrawal,Total_Games_Played,Total_Loyalty_Points,Rank
0,634,515000.0,15737705.0,24,83843.325,1
1,99,1164800.0,2403141.0,10,23665.737,2
2,672,2158700.0,233750.0,10,22757.780,3
3,212,1924981.0,589850.0,1,22199.282,4
4,740,1738490.0,365288.0,2,19211.824,5
5,566,1819175.0,185071.0,183,19153.755,6
6,714,1676300.0,0.0,6,16764.234,7
7,421,878600.0,1269809.0,1557,15446.460,8
8,369,650000.0,1586208.0,37,14438.444,9
9,30,1329000.0,152145.0,13,14053.375,10


# Average Calculations

### 1. Average deposit amount

In [85]:
Avg_deposit = """
-- Calculate the average deposit amount
SELECT AVG(CAST(Amount AS DECIMAL(10, 2))) AS Average_Deposit_Amount
FROM Deposit
WHERE CONVERT(DATE, Datetime) BETWEEN '2022-10-01' AND '2022-10-31';
"""
df= pd.read_sql(Avg_deposit, engine)
print(df)

   Average_Deposit_Amount
0             5492.185399


### 2. Average deposit amount per user

In [90]:
Avg_deposit_per_user = """
-- Calculate the average deposit amount per user
SELECT AVG(User_Deposit_Avg) AS Average_Deposit_Per_User
FROM (
    SELECT 
        User_Id, 
        AVG(CAST(Amount AS DECIMAL(10, 2))) AS User_Deposit_Avg
    FROM 
        Deposit
    WHERE CONVERT(DATE, Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY 
        User_Id
) AS Avg_Deposits;
"""
df = pd.read_sql(Avg_deposit_per_user, engine)
print(df)

   Average_Deposit_Per_User
0               6900.275174


### 3. Average number of games played per user

In [93]:
Avg_game_played= """
-- Calculate the average number of games played per user
SELECT AVG(User_Games_Avg) AS Average_Games_Per_User
FROM (
    SELECT 
        User_ID, 
        AVG(CAST(Games_Played AS INT)) AS User_Games_Avg
    FROM 
        User_Gameplay
    WHERE CONVERT(DATE, Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY 
        User_ID
) AS Avg_Games;
"""
df= pd.read_sql(Avg_game_played, engine)
print(df)

   Average_Games_Per_User
0                       1


# Part B: Bonus Allocation

In [96]:
Bonus_allocation= """
-- Calculate total loyalty points for each player for October 2022
WITH MonthlyDeposits AS (
    SELECT d.User_ID, SUM(CAST(d.Amount AS DECIMAL(10, 2))) AS Total_Deposit, COUNT(*) AS Num_Deposits
    FROM Deposit d
    WHERE CONVERT(DATE, d.Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY d.User_ID
),
MonthlyWithdrawals AS (
    SELECT w.User_ID, SUM(CAST(w.Amount AS DECIMAL(10, 2))) AS Total_Withdrawal, COUNT(*) AS Num_Withdrawals
    FROM Withdrawal w
    WHERE CONVERT(DATE, w.Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY w.User_ID
),
MonthlyGamesPlayed AS (
    SELECT g.User_ID, SUM(CAST(g.Games_Played AS INT)) AS Total_Games_Played
    FROM User_Gameplay g
    WHERE CONVERT(DATE, g.Datetime) BETWEEN '2022-10-01' AND '2022-10-31'
    GROUP BY g.User_ID
),
TotalLoyaltyPoints AS (
    SELECT
        COALESCE(d.User_ID, w.User_ID, g.User_ID) AS User_ID,
        COALESCE(Total_Deposit, 0) AS Total_Deposit,
        COALESCE(Total_Withdrawal, 0) AS Total_Withdrawal,
        COALESCE(Total_Games_Played, 0) AS Total_Games_Played,
        (0.01 * COALESCE(Total_Deposit, 0)) + 
        (0.005 * COALESCE(Total_Withdrawal, 0)) + 
        (0.001 * GREATEST(COALESCE(d.Num_Deposits, 0) - COALESCE(w.Num_Withdrawals, 0), 0)) + 
        (0.2 * COALESCE(Total_Games_Played, 0)) AS Total_Loyalty_Points
    FROM MonthlyDeposits d
    LEFT JOIN MonthlyWithdrawals w ON d.User_ID = w.User_ID
    LEFT JOIN MonthlyGamesPlayed g ON COALESCE(d.User_ID, w.User_ID) = g.User_ID
),
RankedPlayers AS (
    SELECT
        User_ID,
        Total_Loyalty_Points,
        RANK() OVER (ORDER BY Total_Loyalty_Points DESC, Total_Games_Played DESC) AS Player_Rank
    FROM TotalLoyaltyPoints
),
Top50Players AS (
    SELECT 
        User_ID, 
        Total_Loyalty_Points,
        Player_Rank
    FROM RankedPlayers 
    WHERE Player_Rank <= 50
),
TotalPoints AS (
    SELECT SUM(Total_Loyalty_Points) AS Total_Loyalty_Points_Sum 
    FROM Top50Players
)
-- Allocate bonus to top 50 players based on their loyalty points
SELECT 
    p.User_ID,
    p.Total_Loyalty_Points,
    (p.Total_Loyalty_Points / t.Total_Loyalty_Points_Sum) * 50000 AS Bonus_Amount
FROM 
    Top50Players p, 
    TotalPoints t
ORDER BY 
    p.Player_Rank;
"""
df= pd.read_sql(Bonus_allocation, engine)
print(df)

   User_ID  Total_Loyalty_Points  Bonus_Amount
0      634             83843.325       6638.85
1       99             23665.737       1873.85
2      672             22757.780       1802.00
3      212             22199.282       1757.75
4      740             19211.824       1521.20
5      566             19153.755       1516.60
6      714             16764.234       1327.40
7      421             15446.460       1223.05
8      369             14438.444       1143.25
9       30             14053.375       1112.75
10     587             13638.881       1079.95
11     222             13348.803       1056.95
12     352             13040.650       1032.55
13     365             12855.075       1017.85
14     920             12535.800        992.60
15     162             12483.600        988.45
16     415             12304.415        974.25
17     569             12285.423        972.75
18     786             12097.216        957.85
19       2             12040.475        953.35
20     238   

# Part C: Evaluation of the Loyalty Point Formula

#### Fairness of the Current Formula:
The current loyalty point formula is designed to encourage player activity on the platform by rewarding deposits, withdrawals, and the number of games played. While the formula has its strengths, it also has some potential drawbacks in terms of fairness.

##### Current Formula:

##### Loyalty Point = (0.01 * deposit) + (0.005 * Withdrawal amount) + (0.001 * (maximum of (#deposit - #withdrawal) or 0)) + (0.2 * Number of games played)

#### Advantages:

- Encourages players to be consistently active.

- Rewards players for both their financial transactions and gameplay.

#### Limitations:

- It tends to favor players who make large deposits, which could lead to inequality among players.

- The penalty for withdrawals might discourage players from cashing out their winnings, which could be seen as unfair.


#### Suggested Improvements:
##### Equal Points for Deposits and Withdrawals:

Deposit Points: 0.01 × Deposit Amount

Withdrawal Points: 0.01 × Withdrawal Amount

##### Add New Factors:

Loyalty Points: Existing Formula + (0.01 × Number of Days Active)

##### Cap Points from Deposits:

Deposit Points: min(0.01 × Deposit Amount, 100)

##### Balanced Points:

Ensuring both financial transactions and gameplay are equally rewarded.

### Suggested Formula:

##### Loyalty Points =
##### (0.01 × Deposit) + (0.01 × Withdrawal Amount) + (0.001 × max(# deposits - # withdrawals, 0)) + (0.15 × Number of Games Played) + (0.01 × Number of Days Active)

# Conclusion

In conclusion, the loyalty point system implemented by ABC effectively encourages player activity by rewarding financial transactions and gameplay. However, the current formula has some areas that could be improved to ensure greater fairness among players.

**Summary of Findings:**
- The loyalty points are calculated based on deposits, withdrawals, the number of games played, and the difference between deposits and withdrawals.
- Specific slots and overall calculations for October were performed to identify top players.
- Average deposit amounts and game activities were analyzed to provide insights into player behavior.
- Bonus allocation was suggested based on the total loyalty points earned by players.

**Recommendations:**
- Adjust the loyalty point formula to give equal weightage to deposits and withdrawals.
- Introduce additional factors such as duration of membership and activity frequency to reward long-term loyal players.
- Implement a cap on points from deposits to prevent disproportionate rewards.
- Balance the weightage for financial transactions and gameplay to ensure fairness.